In [ ]:
pip install pandas google-cloud-bigquery --upgrade openai

In [ ]:
import pandas as pd
from google.cloud import bigquery
import re
from google.oauth2 import service_account
import openai
import json
import random
import html


# Setup BigQuery client
credentials = service_account.Credentials.from_service_account_file(
    '/path/to/credentials/file')
project_id = 'your-project-id'
client = bigquery.Client(credentials=credentials, project=project_id)

# Setup OpenAI client
openai_client = openai.OpenAI(api_key='your-openai-api-key')

def create_tables_if_not_exist():
    dataset_ref = client.dataset('your-dataset-name')

    # Create guest_profiles table
    guest_profiles_schema = [
        bigquery.SchemaField("guest_name", "STRING", mode="REQUIRED"),
        bigquery.SchemaField("profile", "STRING", mode="REQUIRED"),
    ]
    guest_profiles_table_ref = dataset_ref.table("guest_profiles")
    guest_profiles_table = bigquery.Table(guest_profiles_table_ref, schema=guest_profiles_schema)
    try:
        client.get_table(guest_profiles_table_ref)
    except Exception:
        client.create_table(guest_profiles_table)
        print("Created guest_profiles table.")

    # Create simulated_conversations table
    simulated_conversations_schema = [
        bigquery.SchemaField("topic", "STRING", mode="REQUIRED"),
        bigquery.SchemaField("guests", "STRING", mode="REQUIRED"),
        bigquery.SchemaField("guest_profiles", "STRING", mode="REQUIRED"),
        bigquery.SchemaField("conversation", "STRING", mode="REQUIRED"),
        bigquery.SchemaField("html_conversation", "STRING", mode="REQUIRED"),
    ]
    simulated_conversations_table_ref = dataset_ref.table("simulated_conversations")
    simulated_conversations_table = bigquery.Table(simulated_conversations_table_ref, schema=simulated_conversations_schema)
    try:
        client.get_table(simulated_conversations_table_ref)
    except Exception:
        client.create_table(simulated_conversations_table)
        print("Created simulated_conversations table.")

def get_podcast_data():
    query = """
    SELECT title, transcript, summary_and_insights
    FROM `your-project-id.your-dataset-name.podcast_transcriptions`
    """
    df = client.query(query).to_dataframe()
    print(f"Retrieved {len(df)} podcast episodes.")
    return df

def extract_guest_from_title(title):
    match = re.search(r'with (?:Special Guest|Guests?)\s+(.*?)(?:\s+and\s+|$)', title, re.IGNORECASE)
    if match:
        return [name.strip() for name in re.split(r'\s+and\s+', match.group(1))]
    return []

def identify_guests(df):
    all_guests = []
    for title in df['title']:
        guests = extract_guest_from_title(title)
        all_guests.extend(guests)

    unique_guests = sorted(list(set(all_guests)))
    print(f"Identified {len(unique_guests)} unique guests.")
    return unique_guests

def get_guest_profile_from_bigquery(guest_name):
    query = f"""
    SELECT * FROM `your-project-id.your-dataset-name.guest_profiles`
    WHERE guest_name = '{guest_name}'
    """
    df = client.query(query).to_dataframe()
    if not df.empty:
        return json.loads(df.iloc[0]['profile'])
    return None

def write_guest_profile_to_bigquery(guest_name, profile):
    table_id = 'your-project-id.your-dataset-name.guest_profiles'
    rows_to_insert = [
        {
            'guest_name': guest_name,
            'profile': json.dumps(profile)
        }
    ]
    errors = client.insert_rows_json(table_id, rows_to_insert)
    if errors == []:
        print(f"Profile for {guest_name} written to BigQuery successfully.")
    else:
        print(f"Errors occurred while writing profile for {guest_name}: {errors}")

def generate_guest_profile(guest_name, df):
    profile = get_guest_profile_from_bigquery(guest_name)
    if profile:
        print(f"Profile for {guest_name} retrieved from BigQuery.")
        return profile

    guest_episodes = df[df['title'].str.contains(guest_name, case=False, regex=False)]

    if guest_episodes.empty:
        print(f"No episodes found for guest: {guest_name}")
        return None

    all_transcripts = " ".join(guest_episodes['transcript'].tolist())
    all_summaries = " ".join(guest_episodes['summary_and_insights'].tolist())

    prompt = f"""
    Analyze the following podcast transcripts and summaries for guest {guest_name}.
    Create a brief profile of the guest, including:
    1. Their area of expertise
    2. Their communication style (be specific and colorful in your description)
    3. Key opinions or viewpoints they express (include controversial or unique perspectives)
    4. Any recurring themes or topics they discuss
    5. Their stance on current trends in data analytics or business intelligence
    6. Any memorable quotes or catchphrases they use

    Transcripts: {all_transcripts[:2000]}...

    Summaries: {all_summaries[:2000]}...

    Provide the profile in JSON format with the following keys:
    expertise, communication_style, key_opinions, recurring_themes, trend_stance, memorable_quotes
    """

    response = openai_client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are an AI assistant tasked with creating colorful, opinionated profiles of podcast guests."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=800
    )

    try:
        profile = json.loads(response.choices[0].message.content)
        print(f"Generated profile for guest: {guest_name}")
        write_guest_profile_to_bigquery(guest_name, profile)
        return profile
    except json.JSONDecodeError:
        print(f"Error decoding profile for guest: {guest_name}")
        return None

def generate_topic(df):
    all_titles = " ".join(df['title'].tolist())
    all_summaries = " ".join(df['summary_and_insights'].tolist())

    prompt = f"""
    Based on the following podcast episode titles and summaries, generate an interesting and potentially controversial topic for a new podcast discussion.
    The topic should be related to data analytics, business intelligence, or modern data stack technologies, but should NOT focus on real-time data analytics.
    Choose from a diverse range of subjects such as:
    1. Data governance and ethics
    2. AI and machine learning in business intelligence
    3. The role of data scientists vs. business analysts
    4. Cloud vs. on-premise data solutions
    5. Data visualization techniques and their impact
    6. The future analytics consulting and modern data stack vendors
    7. Data privacy regulations and their effect on analytics
    8. The impact of big data on small businesses
    9. Predictive analytics and its real-world applications
    10. The evolution of data warehousing

    Phrase the topic as a provocative question that would prompt an engaging and opinionated discussion among experts in the field.

    Titles: {all_titles[:1000]}...

    Summaries: {all_summaries[:2000]}...

    Provide the topic as a single question that challenges conventional wisdom or addresses a contentious issue in the field.
    """

    response = openai_client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are an AI assistant tasked with generating diverse and provocative podcast discussion topics about data analytics and business intelligence."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=100
    )

    topic = response.choices[0].message.content.strip()
    print(f"Generated topic: {topic}")
    return topic

def select_relevant_guests(topic, guest_profiles, max_guests=5):
    relevant_guests = []
    for guest, profile in guest_profiles.items():
        relevance_score = sum(word.lower() in topic.lower() for word in profile['expertise'].split())
        relevance_score += sum(word.lower() in topic.lower() for word in profile['recurring_themes'])
        relevant_guests.append((guest, relevance_score))

    relevant_guests.sort(key=lambda x: x[1], reverse=True)
    return [guest for guest, score in relevant_guests[:max_guests]]

def simulate_conversation(topic, guest_profiles, df, num_exchanges=5):
    all_transcripts = " ".join(df['transcript'].tolist())
    questions = re.findall(r"Mark Rittman: (.*?\?)", all_transcripts)
    relevant_questions = [q for q in questions if any(word in q.lower() for word in topic.lower().split())]

    conversation = [f"Mark Rittman: Welcome to the Simulated Drill to Detail Podcast! Today, we're diving into a hot topic: {topic}"]

    selected_guests = select_relevant_guests(topic, guest_profiles)

    for i in range(num_exchanges):
        if relevant_questions and random.random() < 0.7:
            question = random.choice(relevant_questions)
        else:
            prompt = f"""
            Based on the provocative topic "{topic}", generate a challenging follow-up question that Mark Rittman might ask.
            The question should be insightful, potentially controversial, and aimed at sparking an opinionated discussion.
            """
            response = openai_client.chat.completions.create(
                model="gpt-4",
                messages=[
                    {"role": "system", "content": "You are Mark Rittman, the host of a data analytics podcast known for asking tough, thought-provoking questions."},
                    {"role": "user", "content": prompt}
                ],
                max_tokens=100
            )
            question = response.choices[0].message.content.strip()

        conversation.append(f"Mark Rittman: {question}")

        guest = random.choice(selected_guests)
        guest_profile = guest_profiles[guest]

        prompt = f"""
        You are {guest}, a guest on a data analytics podcast. Your expertise is in {guest_profile['expertise']}.
        Your communication style is {guest_profile['communication_style']}.
        Some of your key opinions are: {', '.join(guest_profile['key_opinions'])}.
        Your stance on current trends: {guest_profile['trend_stance']}
        You often use phrases like: {', '.join(guest_profile['memorable_quotes'])}

        Respond to the following question from the host, Mark Rittman:
        "{question}"

        Your response should be:
        1. In your unique communication style
        2. Reflect your expertise and strong opinions
        3. Potentially controversial or challenge conventional wisdom
        4. Include a memorable quote or catchphrase if appropriate
        5. Engaging and thought-provoking

        Aim to provide a response that will spark further discussion or debate.
        """

        response = openai_client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": f"You are {guest}, an opinionated expert in {guest_profile['expertise']}, responding to a podcast question."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=300
        )

        guest_response = response.choices[0].message.content.strip()
        conversation.append(f"{guest}: {guest_response}")

    return conversation, selected_guests

def generate_html_conversation(topic, guests, conversation):
    html_content = f"""
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Simulated Podcast Conversation</title>
    <style>
        body {{ font-family: Arial, sans-serif; line-height: 1.6; max-width: 800px; margin: 0 auto; padding: 20px; }}
        h1 {{ color: #333; }}
        .host {{ color: #0066cc; }}
        .guest {{ color: #006600; }}
        .question, .answer {{ margin-bottom: 20px; }}
    </style>
</head>
<body>
    <h1>{html.escape(topic)}</h1>
    """

    for i, message in enumerate(conversation):
        speaker, content = message.split(': ', 1)
        if speaker == "Mark Rittman":
            html_content += f"""
    <div class="question">
        <p class="host"><strong>{html.escape(speaker)}:</strong> {html.escape(content)}</p>
    </div>
    """
        else:
            html_content += f"""
    <div class="answer">
        <p class="guest"><strong>{html.escape(speaker)}:</strong> {html.escape(content)}</p>
    </div>
    """

    html_content += """
</body>
</html>
    """

    return html_content

def write_conversation_to_bigquery(topic, guests, guest_profiles, conversation):
    table_id = 'ra-development.analytics_seed.simulated_conversations'

    html_conversation = generate_html_conversation(topic, guests, conversation)

    rows_to_insert = [
        {
            'topic': topic,
            'guests': json.dumps(guests),
            'guest_profiles': json.dumps({guest: guest_profiles[guest] for guest in guests}),
            'conversation': json.dumps(conversation),
            'html_conversation': html_conversation
        }
    ]

    errors = client.insert_rows_json(table_id, rows_to_insert)
    if errors == []:
        print(f"Conversation on '{topic}' written to BigQuery successfully.")
    else:
        print(f"Errors occurred while writing conversation: {errors}")

# Main execution
create_tables_if_not_exist()
df = get_podcast_data()
guests = identify_guests(df)

guest_profiles = {}
for guest in guests:
    profile = generate_guest_profile(guest, df)
    if profile:
        guest_profiles[guest] = profile

for _ in range(10):
    topic = generate_topic(df)
    conversation, selected_guests = simulate_conversation(topic, guest_profiles, df)
    write_conversation_to_bigquery(topic, selected_guests, guest_profiles, conversation)

print("Completed generating and storing 10 simulated conversations.")